In [5]:
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K

**data** is the input to the residual module

**K** is the number of filters learnt by the final layer , the first two layers will learn K/4

**stride** parameter controls the stride of convolution and will help reduce spatial dimensions without max pooling

**chanDim** will define the axis which will perform batch normalization

**red** (short for reduce) controls whether we are reducing spatial dimensions or not

**reg** (short for regularization) controls regularization strength applied to the layers

**bnEps** i used to control the ε epsilon which is used to avoid "division by zero" errors while normalizing input

**bnMom** is to control momentum for moving average

In [7]:
'''
data is the input to the residual module
K is the number of filters learnt by the final layer , the first two layers will learn K/4
stride parameter controls the stride of convolution and will help reduce spatial dimensions without max pooling
chanDim will define the axis which will perform batch normalization
red (short for reduce) controls whether we are reducing spatial dimensions or not
reg (short for regularization) controls regularization strength applied to the layers
bnEps i used to control the ε epsilon which is used to avoid "division by zero" errors while normalizing input
bnMom is to control momentum for moving average
'''
class ResNet:
  @staticmethod
  def residual_module(data, K, stride, chanDim, red=False, reg=0.0001, bnEps=2e-5, bnMom=0.9):
    
    #initialize the (identity) shortcut (connection), which is really just a reference to the input data
    shortcut = data

    #CONV layer utilises 1x1 convolutions by K/4 filters. Notice that the bias term is turned off
    bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(data)
    act1 = Activation("relu")(bn1)
    conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act1)

    #for the bottleneck, the second CONV layer will learn K/4 filters that are 3x3
    bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv1)
    act2 = Activation("relu")(bn2)
    conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride,padding="same", use_bias=False, kernel_regularizer=l2(reg))(act2)
    
    #the final block will increase dimension again and will have K 1x1 filters
    bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv2)
    act3 = Activation("relu")(bn3)
    conv3 = Conv2D(K, (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act3)

    #checking if reducing spatial dimensions is necessary so as to avoid max pooling
    #if we have to reduce spatial dimensions then a conv layer with stride>1 will be applied
    if red:
      shortcut = Conv2D(K, (1, 1), strides=stride, use_bias=False, kernel_regularizer=l2(reg))(act1)

    #adding shortcut and final CONV
    x = add([conv3, shortcut])

    #output of the resnet module is x
    return x
  
  #defining build method
  #note that stages and filters parameters are lists. We are stacking N residual modules on top of each other (N=stage value)
  @staticmethod
  def build(width , height, depth, classes, stages, filters, reg=0.0001, bnEps=2e-5, bnMom=0.9):

    #initialize input shape to be channels last and chanDim
    inputShape = (height, width, depth)
    chanDim = -1

    #if using channels first , update input shape and chanDim
    if K.image_data_format() == "channels_first":
      inputShape = (depth, height , width)
      chanDim = 1

    #set input and apply batch normalization
    inputs = Input(shape=inputShape)
    x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(inputs)

    #apply conv->batch normalization->activation->pool to reduce spatial size
    x = Conv2D(filters[0], (5, 5), use_bias=False, padding="same", kernel_regularizer=l2(reg))(x)
    x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
    x = Activation("relu")(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    #start stacking residual layers on top of each other
    #loop over the number of stages
    for i in range(0, len(stages)):
      #initialize stride and apply a res module to reduce spatial size of input volumme
      stride = (1, 1) if i == 0 else (2, 2)
      x = ResNet.residual_module(x, filters[i+1], stride, chanDim, red=True, bnEps=bnEps, bnMom=bnMom)

      #loop over number of layers in stage
      for j in range(0, stages[i]-1):
        #apply resnet module
        x = ResNet.residual_module(x, filters[i+1], (1, 1),chanDim, bnEps=bnEps, bnMom=bnMom)

    #to avoid dense fully connected layers we will apply average pooling to reduce volume size to 1x1xclasses
    #batch normalization->activation->pool
    x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
    x = Activation("relu")(x)
    x = AveragePooling2D((8, 8))(x)

    #finally lets create a dense layer for total classes and then apply softmax activation to generate final probablities as output
    x = Flatten()(x)
    x = Dense(classes, kernel_regularizer=l2(reg))(x)
    x = Activation("softmax")(x)

    #create model
    model = Model(inputs, x, name="resnet")

    # return the final network architecture
    return model

In [8]:
'''
We can now call this class to implement ResNet architecture on any deep learning project
'''

'\nWe can now call this class to implement ResNet architecture on any deep learning project\n'